In [ ]:
# Importing essential libraries

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper-parameters
num_epochs = 6
batch_size = 32
learning_rate = 0.001

# Define data transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a uniform size
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize images
])

# Load dataset
dataset = ImageFolder(root='/kaggle/input/i-naturalist1/inaturalist_12K/train', transform=transform)

# Split dataset into training and validation sets
training_data_size = int(0.8 * len(dataset))
validation_data_size = len(dataset) - training_data_size
training_dataset, validation_dataset = random_split(dataset, [training_data_size, validation_data_size])

# Create data loaders
training_data_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
validation_data_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

#classes = dataset.classes
#print(classes)

In [9]:
# Defining class for CNN
class FlexibleCNN(nn.Module):
    def __init__(self, num_classes, in_channels=3, conv_filters=[16, 32, 64, 128, 256], 
                 kernel_sizes=[3, 3, 3, 3, 3], pool_sizes=[2, 2, 2, 2, 2], 
                 dense_units=512, activation='ReLU', batch_norm = True, dropout = 0.0):
        self.conv_filters = conv_filters
        self.kernel_sizes = kernel_sizes
        self.pool_sizes = pool_sizes
        self.dense_units = dense_units
        self.activation = activation
        self.batch_norm = batch_norm
        self.dropout = dropout
        
        # Invoking the constructor of the base class
        super(FlexibleCNN, self).__init__()

        # Convolutional layers
        self.conv_layers = nn.ModuleList()
        prev_out_size = 224
        in_channels = in_channels
        for out_channels, kernel_size, pool_size in zip(conv_filters, kernel_sizes, pool_sizes):
            conv_layer = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size),
                self.get_activation(activation),
                nn.MaxPool2d(pool_size)
            )
            
            # Making condtion to add the batch norm and dropout
            if batch_norm:
                conv_layer.add_module(f"batch_norm_{out_channels}", nn.BatchNorm2d(out_channels))
            if dropout > 0.0:
                conv_layer.add_module(f"dropout_{out_channels}", nn.Dropout2d(dropout))
            self.conv_layers.append(conv_layer)
            in_channels = out_channels
            
        # Defining function to calculate and return the kernel size dynamically
        def cal_size(stride, padding, kernel_size, prev_size):
            new_size = (prev_size-kernel_size+2*padding)/stride + 1
            return new_size//2
        
        prev_size = 224 # Starting with the size 224
        
        # Running the loop five times for 5 number of blocks
        for i in range(5):
            new_size1 = cal_size(stride = 1, padding = 0, kernel_size = self.kernel_sizes[i], prev_size= prev_size)
            prev_size = new_size1
        print(new_size1)

        #fully connected layers
        #self.fc1 = nn.Linear(conv_filters[-1] * 7 * 7, dense_units)
        self.fc1 = nn.Linear(self.conv_filters[4] * int(new_size1) * int(new_size1), self.dense_units)
        self.fc2 = nn.Linear(self.dense_units, 10)#num_classes)

    def forward(self, x):
        # Forward pass through convolutional layers
        for conv_layer in self.conv_layers:
            x = conv_layer(x)

        # Flatten the output for the fully connected layer
        x = torch.flatten(x, 1)

        # Forward pass through fully connected layers
        #x = F.relu(self.fc1(x))
        x = self.get_activation(self.activation)(self.fc1(x))
        x = self.fc2(x)
        return x
    
    # Defining function to get the different activation functions
    def get_activation(self, activation):
        if activation == 'ReLU':
            return nn.ReLU()
        elif activation == 'GELU':
            return nn.GELU()
        elif activation == 'SiLU':
            return nn.SiLU()
        elif activation == 'Mish':
            return nn.Mish()
        else:
            raise ValueError("Invalid activation function")
            
#model = FlexibleCNN(10)
#print(model)

In [10]:
!pip install wandb  #installing wandb

In [11]:
# Importing wandb and essential libraries
import wandb
import numpy as np
from types import SimpleNamespace
import random

In [12]:
wandb.login(key='cd7a6c2259e8886dc269bbf6f0f9e55089d3beeb')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Defining sweep configuration
sweep_config = {
    'method': 'bayes',
    'name' : 'sweep test23',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'kernel_size':{
            'values': [[3,3,3,3,3], [3,5,5,7,7], [7,7,5,5,3]]
        },
        'dropout': {
            'values': [0.2, 0.3]
        },
        'activation': {
            'values': ['ReLU', 'GELU', 'SiLU', 'Mish']
        },
        'batch_norm':{
            'values': [True,False]
        },
        'filt_org':{
            'values': [[32,32,32,32,32],[128,128,64,64,32],[32,64,128,256,512]]
        },
        'data_augment': {
            'values': [False]
        },
        'num_dense':{
            'values': [128, 256]
        }
    }
}
sweep_id = wandb.sweep(sweep=sweep_config, project='Deep_Learning_Assignment2')

In [ ]:
def main():
    '''
    WandB calls main function each time with differnet combination.

    We can retrive the same and use the same values for our hypermeters.

    '''


    with wandb.init(entity = 'prabhat-kumar') as run:

        run_name="-act_"+wandb.config.activation+"-ks"+str(wandb.config.kernel_size)+'-nd'+str(wandb.config.num_dense)
        wandb.run.name=run_name
#         actv = get_activation(wandb.config.activation)
        model = FlexibleCNN(num_classes=10, in_channels=3, conv_filters=wandb.config.filt_org, 
                 kernel_sizes=wandb.config.kernel_size, pool_sizes=[2, 2, 2, 2, 2], 
                 dense_units=wandb.config.num_dense, activation=wandb.config.activation).to(device)
        
        
        # Loss and optimizer
        criterion = nn.CrossEntropyLoss()
        #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)   # Using Adam as an optimizer

        # Training loop
        for epoch in range(num_epochs):
            #initializing the necessary values
            total_training_data = 0
            correct_predicted_train_data = 0
            loss1 = 0
            # Train the model
            model.train()
            for images, labels in training_data_loader:
                # Moving images and labels to GPU
                images = images.to(device)
                labels = labels.to(device)
        
                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)
        
                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                loss1 = loss1 + loss.item()
                
                # Calculate the training accuracy
                max_value, predicted_by_model = torch.max(outputs, 1)
                total_training_data = total_training_data + labels.size(0)
                correct_predicted_train_data = correct_predicted_train_data + (predicted_by_model == labels).sum().item()
                
            train_accuracy = 100*correct_predicted_train_data/total_training_data
            print(f'Epoch number {epoch+1}, Training Accuracy: {train_accuracy:.2f}%') # Printing training accuracy
            wandb.log({'train_loss': round(loss1 / len(training_data_loader), 2)})
            wandb.log({'train_accuracy': round(train_accuracy, 2)})

            #wandb.log({'train_loss':running_loss/len(train_loader)})
            #wandb.log({'train_accuracy':train_accuracy})
            
        
    
            # Validate the model
            model.eval()
            with torch.no_grad():
                # Initializing the values
                correct_predicted_validation_data = 0
                total_validation_data = 0
                for images, labels in validation_data_loader:
                    
                    images = images.to(device)  # Moving images to GPU
                    labels = labels.to(device)  # Moving labels to GPU
            
                    # Forward pass
                    outputs = model(images)
                    max_value, predicted_by_model = torch.max(outputs, 1)
            
                    # Compute accuracy
                    total_validation_data = total_validation_data + labels.size(0)
                    correct_predicted_validation_data = correct_predicted_validation_data + (predicted_by_model == labels).sum().item()
            
        
                val_accuracy = 100 * correct_predicted_validation_data / total_validation_data
                print(f'Epoch number {epoch+1}, Validation Accuracy: {val_accuracy:.2f}%')
                wandb.log({'val_accuracy':val_accuracy})
                wandb.log({'epoch':epoch+1})
    

        print('Finished Training') # Here training finished

wandb.agent(sweep_id, function=main,count=1) # calls main function for count number of times.
wandb.finish()

## Testing using best hyperparameter

In [8]:
# Importing essential libraries

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Hyper-parameters
num_epochs = 5
batch_size = 32
learning_rate = 0.001

# Define data transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a uniform size
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize images
])

# Load dataset
dataset = ImageFolder(root='/kaggle/input/i-naturalist1/inaturalist_12K/train', transform=transform)

# Split dataset into training and validation sets
training_data_size = int(0.8 * len(dataset))
validation_data_size = len(dataset) - training_data_size
training_dataset, validation_dataset = random_split(dataset, [training_data_size, validation_data_size])

# Create data loaders
training_data_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
validation_data_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

testing_dataset = ImageFolder(root='/kaggle/input/i-naturalist1/inaturalist_12K/val', transform=transform)

testing_data_loader = DataLoader(testing_dataset, batch_size=batch_size, shuffle=True)

#classes = dataset.classes
#print(classes)

In [13]:
sweep_config = {
    'method': 'bayes',
    'name' : 'sweep test(test_data)1',
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'kernel_size':{
            'values': [[3,5,5,7,7]]
        },
        'dropout': {
            'values': [0.3]
        },
        'activation': {
            'values': ['GELU']
        },
        'batch_norm':{
            'values': [False]
        },
        'filt_org':{
            'values': [[32,64,128,256,512]]
        },
        'data_augment': {
            'values': [False]
        },
        'num_dense':{
            'values': [128]
        }
    }
}
sweep_id = wandb.sweep(sweep=sweep_config, project='Deep_Learning_Assignment2')

Create sweep with ID: y48kx8hl
Sweep URL: https://wandb.ai/prabhat-kumar/Deep_Learning_Assignment2/sweeps/y48kx8hl


In [ ]:
def main():
    '''
    WandB calls main function each time with differnet combination.

    We can retrive the same and use the same values for our hypermeters.

    '''


    with wandb.init(entity = 'prabhat-kumar') as run:

        run_name="-act_"+wandb.config.activation+"-ks"+str(wandb.config.kernel_size)+'-nd'+str(wandb.config.num_dense)
        wandb.run.name=run_name
#         actv = get_activation(wandb.config.activation)
        model = FlexibleCNN(num_classes=10, in_channels=3, conv_filters=wandb.config.filt_org, 
                 kernel_sizes=wandb.config.kernel_size, pool_sizes=[2, 2, 2, 2, 2], 
                 dense_units=wandb.config.num_dense, activation=wandb.config.activation).to(device)
        
        
        # Loss and optimizer
        criterion = nn.CrossEntropyLoss()
        #optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)   # Using Adam as an optimizer

        # Training loop
        for epoch in range(num_epochs):
            #initializing the necessary values
            total_training_data = 0
            correct_predicted_train_data = 0
            loss1 = 0
            # Train the model
            model.train()
            for images, labels in training_data_loader:
                # Moving images and labels to GPU
                images = images.to(device)
                labels = labels.to(device)
        
                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)
        
                # Backward and optimize
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                loss1 = loss1 + loss.item()
                
                # Calculate the training accuracy
                max_value, predicted_by_model = torch.max(outputs, 1)
                total_training_data = total_training_data + labels.size(0)
                correct_predicted_train_data = correct_predicted_train_data + (predicted_by_model == labels).sum().item()
                
            train_accuracy = 100*correct_predicted_train_data/total_training_data
            print(f'Epoch number {epoch+1}, Training Accuracy: {train_accuracy:.2f}%') # Printing training accuracy
            wandb.log({'train_loss': round(loss1 / len(training_data_loader), 2)})
            wandb.log({'train_accuracy': round(train_accuracy, 2)})

            #wandb.log({'train_loss':running_loss/len(train_loader)})
            #wandb.log({'train_accuracy':train_accuracy})
            
        
    
            # Validate the model
            model.eval()
            with torch.no_grad():
                # Initializing the values
                correct_predicted_testing_data = 0
                total_testing_data = 0
                for images, labels in testing_data_loader:
                    
                    images = images.to(device)  # Moving images to GPU
                    labels = labels.to(device)  # Moving labels to GPU
            
                    # Forward pass
                    outputs = model(images)
                    max_value, predicted_by_model = torch.max(outputs, 1)
            
                    # Compute accuracy
                    total_testing_data = total_testing_data + labels.size(0)
                    correct_predicted_testing_data = correct_predicted_testing_data + (predicted_by_model == labels).sum().item()
            
        
                test_accuracy = 100 * correct_predicted_testing_data / total_testing_data
                print(f'Epoch number {epoch+1}, Test Accuracy: {test_accuracy:.2f}%')
                wandb.log({'test_accuracy':test_accuracy})
                wandb.log({'epoch':epoch+1})
    

        print('Finished Training') # Here training finished

wandb.agent(sweep_id, function=main,count=1) # calls main function for count number of times.
wandb.finish()